In [19]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, RandomTreesEmbedding, VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_union, make_pipeline
from sklearn.preprocessing import FunctionTransformer,OneHotEncoder
from xgboost import XGBClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression

In [72]:
datadir= '/home/db2inst1/task/'
df_train = pd.read_csv( datadir + 'trainData.csv')   
df_test = pd.read_csv( datadir + 'testData.csv')
df_s = df_train.sample(frac=0.5,random_state=5)

feature_cols = df_train.columns[0:-1]
target_col = df_train.columns[-1]

X_tr=df_train[feature_cols].values
y_tr=df_train[target_col].values-1

X=df_s[feature_cols].values
y=df_s[target_col].values-1

X_test=df_test[feature_cols].values

# categorical features creation

In [25]:
transformation = make_union(
    FunctionTransformer(),
    RandomTreesEmbedding(),
     make_pipeline(
         FunctionTransformer(func=lambda x: (x.round(1) * 10)),
         OneHotEncoder()
     )
)
model = make_pipeline(
    transformation,
    VotingClassifier(
        [
            ('LR', LogisticRegression(multi_class='multinomial',n_jobs=-1,solver='newton-cg')),
            ('sgd', GridSearchCV(SGDClassifier(loss='log',tol=1e-3,max_iter=1000,n_jobs=-1),
                                 param_grid={'alpha': [0.1, 0.03, 0.01, 0.003],
                                            'class_weight':['balanced',None]
                                             #'loss':['log','hinge']
                                            },
                                 scoring='neg_log_loss',
                                 n_jobs=-1)),
            ('rf1',
             RandomForestClassifier(n_estimators=700,
                                    criterion="gini", max_features=20,                              
                                    min_samples_split=7, n_jobs=-1)),
            
            ('mlp1', MLPClassifier()),
            ('mlp2', MLPClassifier(hidden_layer_sizes=(50,))),
            ('xgb', XGBClassifier(max_depth=7, nthread=6,n_jobs=-1)),
            ('gsearch2', GridSearchCV(XGBClassifier( learning_rate=0.1, 
  gamma=0, colsample_bytree=0.8,
 objective= 'multi:softprob', scale_pos_weight=1,seed=27), 
 param_grid = param_test2, scoring='neg_log_loss',n_jobs=-1,iid=False, cv=5)
            )], 
             voting='soft'
    )
)